# MULTI LAYER PERCEPTRON

The following code is for a Multilayer perceptron.I have used feed forward,gradient descent and error backpropagation algorithm to model the network.

In [1]:
from pandas import DataFrame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#list of activation functions
def sigmoid(x):
        return 1/(1+np.exp(-x))
def linear(x):
        return x
def tanh(x):
        return ((np.exp(2*x)-1)/(np.exp(2*x)+1))

In [3]:
class MLP:
    #initialising the dataset
    def __init__(self,inodes,hnodes,onodes,h_actf,o_actf):
        self.inodes = inodes
        self.hnodes = hnodes
        self.onodes = onodes
        self.h_actf = h_actf
        self.o_actf = o_actf
        self.input_weights = np.random.random((hnodes,inodes))
        self.hidden_weights = np.random.random((onodes,hnodes))
    #training the dataset
    def train(self,data,eta,epoch):
        for i in range(epoch):
            for i in range(len(data.index)):
                #setting the input and target values
                inputs=np.array(data.ix[i][:self.inodes]).reshape((self.inodes,1))
                targets=np.array(data.ix[i,self.inodes:]).reshape((self.onodes,1))
                #finding the outputs of hidden layer
                if self.h_actf=='sigmoid':
                    hidden_outputs=sigmoid(np.dot(self.input_weights,inputs))
                elif self.h_actf=='tanh':
                    hidden_outputs=tanh(np.dot(self.input_weights,inputs))
                elif self.h_actf=='linear':
                    hidden_outputs=linear(np.dot(self.input_weights,inputs))
                #finding the outputs of target layer
                if self.o_actf=='sigmoid':
                    final_outputs=sigmoid(np.dot(self.hidden_weights,hidden_outputs))
                elif self.o_actf=='tanh':
                    final_outputs=tanh(np.dot(self.hidden_weights,hidden_outputs))
                elif self.o_actf=='linear':
                    final_outputs=linear(np.dot(self.hidden_weights,hidden_outputs))
                #errors of target and hidden layer
                error_target = targets - final_outputs
                error_hidden = np.dot(np.transpose(self.hidden_weights),error_target)
                #updating the weights of hidden_target layer
                if self.o_actf=='sigmoid':
                    self.hidden_weights += eta*np.dot((error_target*final_outputs*(1-final_outputs)),np.transpose(hidden_outputs))
                elif self.o_actf=='tanh':
                    self.hidden_weights += eta*np.dot((error_target*(1-final_outputs**2)),np.transpose(hidden_outputs))
                elif self.o_actf=='linear':
                    self.hidden_weights += eta*np.dot(error_target,np.transpose(hidden_outputs))
                #updating the weights of input_hidden layer
                if self.h_actf=='sigmoid':
                    self.input_weights += eta*np.dot((error_hidden*hidden_outputs*(1-hidden_outputs)),np.transpose(inputs))
                elif self.h_actf=='tanh':
                    self.input_weights += eta*np.dot((error_hidden*(1-hidden_outputs**2)),np.transpose(inputs))
                elif self.h_actf=='linear':
                    self.input_weights += eta*np.dot(error_hidden,np.transpose(inputs))
    #testing the dataset           
    def test(self,data,problem):
        classify=[]
        for i in range(len(data.index)):
                inputs=np.array(data.ix[i][:self.inodes]).reshape((self.inodes,1))
                #finding the outputs of hidden layer
                if self.h_actf=='sigmoid':
                    hidden_outputs=sigmoid(np.dot(self.input_weights,inputs))
                elif self.h_actf=='tanh':
                    hidden_outputs=tanh(np.dot(self.input_weights,inputs))
                elif self.h_actf=='linear':
                    hidden_outputs=linear(np.dot(self.input_weights,inputs))
                #finding the outputs of target layer
                if self.o_actf=='sigmoid':
                    final_outputs=sigmoid(np.dot(self.hidden_weights,hidden_outputs))
                elif self.o_actf=='tanh':
                    final_outputs=tanh(np.dot(self.hidden_weights,hidden_outputs))
                elif self.o_actf=='linear':
                    final_outputs=linear(np.dot(self.hidden_weights,hidden_outputs))
                #output for a classification problem
                if problem =='classification':
                    #print('output is',np.argmax(final_outputs))
                    classify.append(np.argmax(final_outputs))
                #output for a regression problem
                if problem == 'regression':
                    #print('output is',final_outputs[0])
                    classify.append(final_outputs[0])
        return classify   

In [4]:
##creating a data for XOR gate
xor_gate=DataFrame(np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]]),columns=['input1','input2','target'])
xor_gate

,input1,input2,target
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [5]:
##transforming the data for XOR gate to feed into the MLP
xor_gate=DataFrame(np.array([[0,0,1,0],[0,1,0,1],[1,0,0,1],[1,1,1,0]]),columns=['input1','input2','target_0','target_1'])
xor_gate

,input1,input2,target_0,target_1
0,0,0,1,0
1,0,1,0,1
2,1,0,0,1
3,1,1,1,0


In [6]:
#initialising and training the data
p = MLP(2,2,2,'tanh','tanh')
p.train(xor_gate,0.25,100)

In [7]:
#testing the data
p.test(xor_gate,'classification')

[0, 1, 1, 0]

Output seems to match with the target values

In [ ]:
#classifying the iris dataset
#I have already splitted the dataset into training,validation and testing dataset.

In [8]:
train=pd.read_csv('C:/Users/HP/Desktop/datasets/training.csv')
test=pd.read_csv('C:/Users/HP/Desktop/datasets/testing.csv')
validate=pd.read_csv('C:/Users/HP/Desktop/datasets/validate.csv')

In [9]:
#dropping the species column
train_final=train.drop('Species',axis=1)
test_final=test.drop('Species',axis=1)
validate_final=validate.drop('Species',axis=1)

In [10]:
#initialising and training the network
p = MLP(4,4,3,'sigmoid','sigmoid')
p.train(train_final,0.3,500)

In [11]:
#accuracy for validation set
x=p.test(validate_final,'classification')
cls=x
count=0
for i in range(len(x)):
    if x[i]==0:
        cls[i]='setosa'
    elif x[i]==1:
        cls[i]='versicolor'
    else:
        cls[i]='virginica'
orig=validate['Species'].values
for i in range(len(orig)):
    if orig[i]==cls[i]:
        count +=1
accuracy=count/len(orig)
accuracy

0.8947368421052632

In [12]:
#accuracy for testing dataset
x=p.test(test_final,'classification')
cls=x
count=0
for i in range(len(x)):
    if x[i]==0:
        cls[i]='setosa'
    elif x[i]==1:
        cls[i]='versicolor'
    else:
        cls[i]='virginica'
orig=test['Species'].values
for i in range(len(orig)):
    if orig[i]==cls[i]:
        count +=1
accuracy=count/len(orig)
accuracy

0.7027027027027027

A lot of improvement needs to be done on our MLP model,like tuning the learning rate,epochs or adjusting the 
no of hidden nodes.There are ways to find the optimal values of these parameters which is a topic for some other time.
But for now you can try by changing the activation functions and see how the accuracy changes.